1. Сравните метрики hit_rate@k, precision@k.   
Какую метрику использовать предпочтительно и почему?  
Приведите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?


Ответ: hit_rate@k бинарная, и отражает лишь факт "попадания", хотя бы минимального (или не попадания) в наших топ k рекомендациях. Т.е. эта метрика даст одинаковый результат (1), если рекомендации будут минимально точные и если рекомендации будут максимадьно точные (т.е. не будет разницы, один ли товар из рекомендованных куплен или все k)

precision@k более "гибкая" и отражает не только положительный (или нулевой) результат рекомендаций, но и уровень (или долю) "попаданий" в k рекомендациях. Полагаю, в большинстве задач эта метрика предпочтительнее, т.к. она позволяет лучше настроить модель и позволяет сравнивать результат разных моделей, в которых hit_rate даст одинаковый score.

Технически, hit_rate предпочтительнее в случае рекомендации одного товара (k=1), в этом случае результат hit_rate@k одинаков с precision@k :

например какое-нибуть one time offer (когда клиент - посетитель сайта гикбрейнс, и ему предлагается подписка на рекомендованный курс со скидкой, которая действует в течение короткого времени)

Также, полагаю, hit_rate может быть предпочтительнее в тех  случаях, когда для бизнеса достаточно, чтобы хотя бы одна рекомендация сработала (продаваемый товар покупается редко и/или он достаточно дорогой:

клиент - человек, покупающий жилье, товар - дом/квартира;
клиент - водитель, товар - автомобиль;
клиент - посетитель сайта с дорогой мебелью, товар - диван или кухонный гарнитур


2. В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Приведите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в  логарифм.



In [1]:
import numpy as np

$DCG@5 = \frac{1}{5}*(1 / 1 + 0 / 2 + 0 / log(3) + 1 / log(4) + 0 / log(5))$  
$ideal DCG@5 = \frac{1}{5}*(1 / 1 + 1 / 2 + 1 / log(3) + 1 / log(4) + 1 / log(5))$  

$NDCG = \frac{DCG}{ideal DCG}$

In [3]:
np.log2(3)

1.584962500721156

Знаменатель корректирует вес релевантных значений в зависимости от их "положения". По логике вес должен уменьшаться с увеличением "номера" в ранжировании.

Т.к. log(i)< i , то думаю, что логарифм используются для более плавного уменьшения влияния на результат рекомендаций, находящмихся после 1й и 2й. 

При этом, мне не совсем понятно, почему для 3го значения "дисконт" меньше чем для второго ( log2(3)~1,58 ), и для 4го значения знаменатель (log2(4)) такой же как для второго. И лишь с 5го значения вес уменьшается с каждой позицией.

По такой логике вместо логарифма можно использовать какую-нибудь другую функцию, уменьшающую "скорость" увеличения (например степенную функцию с значением степени меньше 1 (1/3 например))

3. Какие еще метрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики **и чем являются интеракции**, чтобы она могла быть использована).

Mean average recall @k (MAR@K). Аналогична MAP@K, вместо precision используется recall

WTA (winner takes all) – эта метрика равна 1, если топ-рекомендация (с самым большим предсказанным рейтингом) из просмотренных пользователем получила положительную оценку, и 0 в противном случае;



4. boughted = [1, 3, 5, 7, 9, 11]  
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]  
Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8 (expected reciprocal rank)  
Сначала поссчитайет "руками", а затем подтвердите рассчеты с помощью функций (некоторые были на семинаре, некоторые нужно будут написать самостоятельно)

ERR@8 нашел на https://habr.com/ru/company/econtenta/blog/303458/

    к сожалению, не понял эту метрику и как она считается
    
    
    Expected reciprocal rank (ERR) — пример метрики качества ранжирования, основанной на каскадной модели. Задается она следующей формулой и т.д. по ссылке

In [14]:
def precision_at_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

def recall_at_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall



In [31]:
def ap_k(recommended_list, bought_list, k=8):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list[:k])
    
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0
    
    sum_ = 0
    for i in range(1, k+1):
        
        if flags[i-1] == True:
            p_k = precision_at_k(recommended_list, bought_list, k=i)
            sum_ += p_k
            
    result = sum_ / sum(flags)
    
    return result

In [33]:
def ndsg_k(recommended_list, bought_list, k=8):
    
    recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list, bought_list)
    
    if sum(flags) == 0:
        return 0

    dsg_k = 0
    idealdsg = 0
    for i, flag in enumerate (flags):
        
        if i<2:
            if flag == True:
                dsg_k += flag/(i+1)
            idealdsg += 1/(i+1)
        else:
            if flag == True:
                dsg_k += flag/np.log2(i+1)
            idealdsg += 1/np.log2(i+1)
                
    # по итогам расчета можно разделить dsg_k и idealdsg на k, но на результат это не влияет, поэтому не стал делить
    return dsg_k/idealdsg

def rr_k(recommended_list, bought_list, k=8):
    
    recommended_list = recommended_list[:k]
    flags = np.isin(recommended_list, bought_list)
    
    sum = 0
    for i, flag in enumerate (flags):
        
        sum += flag/(i+1)
    
    return sum/k
    

In [23]:
bought_list = [1, 3, 5, 7, 9, 11]  
recommended_list = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3] 

In [24]:
precision_at_k(recommended_list, bought_list, k=8)

0.5

In [25]:
recall_at_k(recommended_list, bought_list, k=8)

0.6666666666666666

In [32]:
ap_k(recommended_list, bought_list, k=8)

0.6083333333333333

In [34]:
ndsg_k(recommended_list, bought_list, k=8)

0.4708698136085946

In [30]:
rr_k(recommended_list, bought_list, k=8)

0.15

In [35]:
#для ручного расчета ndcg

sum0 = 0.5
sum1 = 1.5
for i in range(3,9):
    a = 1/np.log2(i)
    print(a)
    if i in [3,5,6]:
        sum0 += a
    sum1 += a
print(sum0,sum1, sum0/sum1)

0.6309297535714575
0.5
0.43067655807339306
0.38685280723454163
0.3562071871080222
0.3333333333333333
1.9484591188793923 4.137999639320748 0.4708698136085946
